# ⚛️ LiH — Ground-State VQE vs Bond Length (Noiseless)

This notebook performs a **bond-length scan** for **lithium hydride (LiH)** using
**noiseless VQE** with a **UCC doubles** ansatz:

- Sweep Li–H distances: **1.1 Å → 2.1 Å**
- Build the LiH Hamiltonian at each geometry (STO-3G)
- Optimize a UCC-Doubles ansatz with **Gradient Descent**
- Extract:
  - **Energy vs bond length curve**
  - **Approximate equilibrium bond length**

This notebook uses the same conventions as the other VQE notebooks.

In [ ]:
import pennylane as qml
from pennylane import numpy as np
from pennylane import qchem
import matplotlib.pyplot as plt

import sys, os, json, time
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))

from vqe.vqe_utils import (
    IMG_DIR,
    excitation_ansatz,
    get_optimizer,
    set_seed,
    make_run_config_dict,
    run_signature,
    find_existing_run,
    save_run_record,
    ensure_dirs,
    build_run_filename,
)

ensure_dirs()

# 🧬 Part 1 — Global Configuration

- Molecule: LiH  
- Basis: STO-3G  
- Electrons: 4  
- Ansatz: UCC doubles  
- Optimizer: Gradient Descent  

We use the same seed + config logic as other notebooks.

In [ ]:
seed = 0
set_seed(seed)

symbols = ["Li", "H"]
basis = "STO-3G"
electrons = 4

bond_lengths = np.linspace(1.1, 2.1, 10)

ansatz_desc = "UCC doubles"
optimizer_name = "GradientDescent"
stepsize = 0.8
max_iterations = 25

# 🧱 Part 2 — LiH Geometry Builder

In [ ]:
def lih_geometry(bond_length):
    """Return LiH atomic coordinates (Å)."""
    return np.array([
        [0.0, 0.0, 0.0],          # Li
        [0.0, 0.0, bond_length],  # H
    ])

# ⚙️ Part 3 — Noiseless VQE QNode Factory  

We construct a **geometry-specific** VQE QNode where:

- HF state, doubles excitations, and Hamiltonian depend on the bond length  
- The device is noiseless (`default.qubit`)
- Differentiation uses `parameter-shift`

In [ ]:
def make_vqe_qnode(hamiltonian, hf_state, doubles, num_wires):
    """Return a noiseless QNode for UCC doubles at a fixed geometry."""
    dev = qml.device("default.qubit", wires=num_wires)

    @qml.qnode(dev, diff_method="parameter-shift")
    def circuit(params):
        excitation_ansatz(
            params,
            wires=range(num_wires),
            hf_state=hf_state,
            excitations=doubles,
            excitation_type="double",
        )
        return qml.expval(hamiltonian)

    return circuit

# 💾 Part 4 — Cache-Aware VQE per Bond Length  

Each geometry:
- builds LiH Hamiltonian  
- builds HF state  
- builds UCC doubles ansatz  
- checks for cached run  
- performs VQE if needed  

This matches your caching style across all molecules.

In [ ]:
def run_vqe_for_bond_length(bond_length):
    """Perform VQE for LiH at a fixed bond length, with caching."""
    # --- Build geometry and Hamiltonian ---
    coordinates = lih_geometry(bond_length)
    hamiltonian, qubits = qchem.molecular_hamiltonian(
        symbols, coordinates, charge=0, basis=basis
    )

    hf = qchem.hf_state(electrons, qubits)
    _, doubles = qchem.excitations(electrons, qubits)

    # --- Build config hash (includes geometry) ---
    cfg = make_run_config_dict(
        symbols=symbols,
        coordinates=coordinates,
        basis=basis,
        ansatz_desc=ansatz_desc,
        optimizer_name=optimizer_name,
        stepsize=stepsize,
        max_iterations=max_iterations,
        seed=seed,
    )
    cfg["bond_length"] = float(bond_length)

    sig = run_signature(cfg)
    fname = build_run_filename(
        f"LiH_Bond_{bond_length:.3f}A",
        optimizer_name,
        seed,
        sig,
    )
    existing = find_existing_run(sig)

    # --- Reuse cached run ---
    if existing:
        with open(existing) as f:
            rec = json.load(f)
        energy_curve = rec["energies"]
        final_energy = float(energy_curve[-1])
        return final_energy, energy_curve

    # --- Fresh run ---
    cost_fn = make_vqe_qnode(hamiltonian, hf_state=hf, doubles=doubles, num_wires=qubits)
    opt = get_optimizer(optimizer_name, stepsize=stepsize)

    params = np.zeros(len(doubles))
    energy_curve = [cost_fn(params)]

    for _ in range(max_iterations):
        params, e_prev = opt.step_and_cost(cost_fn, params)
        energy_curve.append(e_prev)

    # Recompute final point
    energy_curve[-1] = float(cost_fn(params))
    final_energy = float(energy_curve[-1])

    # Save to disk
    rec = {
        "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
        "molecule": "LiH",
        "symbols": symbols,
        "geometry": coordinates.tolist(),
        "basis": basis,
        "electrons": electrons,
        "num_wires": qubits,
        "ansatz": ansatz_desc,
        "optimizer": {
            "name": optimizer_name,
            "stepsize": stepsize,
            "iterations_planned": max_iterations,
            "iterations_ran": len(energy_curve) - 1,
        },
        "seed": seed,
        "energies": [float(e) for e in energy_curve],
        "final_params": [float(x) for x in params],
        "config_hash": sig,
        "metadata": {"bond_length_A": float(bond_length)},
    }

    persisted = save_run_record(fname, rec)
    print(f"[saved] {fname}")
    print(f"[mirrored] {persisted}")

    return final_energy, energy_curve

# 🚀 Part 5 — Sweep Bond Lengths

In [ ]:
energies = []
curves = {}

for r in bond_lengths:
    E_final, E_curve = run_vqe_for_bond_length(r)
    energies.append(E_final)
    curves[r] = E_curve
    print(f"r = {r:.2f} Å → E = {E_final:.8f} Ha")

energies = np.array(energies)

# 📉 Part 6 — Energy vs Bond Length

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(bond_lengths, energies, marker="o")
plt.xlabel("Li–H bond length (Å)")
plt.ylabel("Ground-state energy (Ha)")
plt.title("LiH VQE: Energy vs Bond Length (Noiseless, UCC Doubles)")
plt.grid(True)
plt.tight_layout()
plt.savefig(f"{IMG_DIR}/LiH_Bond_Length_Scan.png", dpi=300)
plt.show()

# 📌 Part 7 — Extract Approximate Equilibrium Bond Length

In [ ]:
min_index = int(np.argmin(energies))
optimal_r = bond_lengths[min_index]
optimal_E = energies[min_index]

print(f"\nEstimated equilibrium bond length: {optimal_r:.2f} Å")
print(f"Minimum VQE ground-state energy: {optimal_E:.8f} Ha")